In [12]:
import requests
import lxml.html
import pprint

from delete_database_entries import *

## Note: start your local server!

In the django project directory (where the manage.py file is located) type: "python manage.py runserver"

## Deleting all entries in the database to make a clean start

To keep the database clean we delete all content first :O

In [13]:
delete_models("observations")
delete_models("sessions")
delete_models("pipelineconfigurations")

Deleting: observations
6
7
8
Deleting: sessions
Deleting: pipelineconfigurations
5
6


## We first make an observation by posting it to the server

You should imagine this observation object simply has a name and sort of an identifier that points to a database actually containing the observation data

In [14]:
obs_name = "pipodeclouwnnebula"
with requests.Session() as s:
    response = s.post('http://localhost:8000/observations/', data={"name": obs_name, "some_identifier": "a catalogue number of some kind"})
    print("Response code: ", response.status_code)
    pp.pprint(response.json())
    obs_id = response.json()["id"]

Response code:  201
{   'id': 9,
    'name': 'pipodeclouwnnebula',
    'some_identifier': 'a catalogue number of some kind'}


## Now we make a pipeline configuration object

You should imagine this contains all the parameters for a specific pipeline that need to be set to run the pipeline

In [15]:
pipelineconf_name = "pipeline setup for clown nebula"
with requests.Session() as s:
    response = s.post('http://localhost:8000/pipelineconfigurations/', data={"name": pipelineconf_name, "some_setting_1": 1.5, "some_setting_2": 3.6 })
    print("Response code: ", response.status_code)
    pp.pprint(response.json())
    pl_conf_id = response.json()["id"]

Response code:  201
{   'id': 7,
    'name': 'pipeline setup for clown nebula',
    'some_setting_1': 1.5,
    'some_setting_2': 3.6}


## We can now make a session object

Here imagine that the session object is an object that will actually run your pipeline configuration on your data. It contains direct links (through primary keys) to an observation and pipelineconfiguration object

In [16]:
with requests.Session() as s:
    response = s.post('http://localhost:8000/sessions/', data={"name": "run 1 for clown nebula", "observation": obs_id, "pipeline_conf": pl_conf_id })
    print("Response code: ", response.status_code)
    pp.pprint(response.json())

Response code:  201
{   'date_created': '2018-05-16T08:55:55.273186Z',
    'date_modified': '2018-05-16T08:55:55.273186Z',
    'id': 6,
    'name': 'run 1 for clown nebula',
    'observation': 9,
    'pipeline_conf': 7}


## Now lets have a look at the content of the database through some get requests

### Sessions:

In [17]:
response = s.get('http://localhost:8000/sessions/')
pp.pprint(response.json())

[   {   'date_created': '2018-05-16T08:55:55.273186Z',
        'date_modified': '2018-05-16T08:55:55.273186Z',
        'id': 6,
        'name': 'run 1 for clown nebula',
        'observation': 9,
        'pipeline_conf': 7}]


### Observations:

In [18]:
response = s.get('http://localhost:8000/observations/')
pp.pprint(response.json())

[   {   'id': 9,
        'name': 'pipodeclouwnnebula',
        'some_identifier': 'a catalogue number of some kind'}]


### PipelineConfigurations: 

In [19]:
response = s.get('http://localhost:8000/pipelineconfigurations/')
pp.pprint(response.json())

[   {   'id': 7,
        'name': 'pipeline setup for clown nebula',
        'some_setting_1': 1.5,
        'some_setting_2': 3.6}]
